In [11]:
# Let's import Keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split


Dataset Preparation and Pre-Processing

In [12]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import keras
from keras import backend as K

xdata = []
ydata = []

# Dictionary to map piece names to numeric labels
piece_to_label = {}
label_counter = 0

# Target size will be 1/8 of original (divide both dimensions by 8)
SCALE_FACTOR = 8

for colour in os.listdir("chess_png"):
    if colour.startswith('.'):
        continue
    for piece in os.listdir(os.path.join("chess_png", colour)):
        if piece.startswith('.'):
            continue
        
        piece_name = f"{colour}_{piece}"
        piece_path = os.path.join("chess_png", colour, piece)
        
        # Skip if not a directory
        if not os.path.isdir(piece_path):
            continue
        
        # Assign numeric label to this piece type
        if piece_name not in piece_to_label:
            piece_to_label[piece_name] = label_counter
            label_counter += 1
        
        for image in os.listdir(piece_path):
            if image.startswith('.'):
                continue
            
            src = os.path.join(piece_path, image)
            img = cv2.imread(src, cv2.IMREAD_GRAYSCALE)
            
            # Check if image was read successfully
            if img is None:
                print(f"Warning: Could not read {src}")
                continue
            
            # Resize to 1/8 of original size
            new_height = img.shape[0] // SCALE_FACTOR
            new_width = img.shape[1] // SCALE_FACTOR
            img_resized = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
            
            ydata.append(piece_to_label[piece_name])  # Use numeric label
            xdata.append(img_resized)

# Get image dimensions from the first image
img_rows, img_cols = xdata[0].shape

# Convert to numpy arrays
xdata = np.array(xdata)
ydata = np.array(ydata)

print(f"Loaded {len(xdata)} images")
print(f"Piece types: {piece_to_label}")
print(f"Original image dimensions (estimated): {img_rows * SCALE_FACTOR}x{img_cols * SCALE_FACTOR}")
print(f"Resized image dimensions: {img_rows}x{img_cols}")

RANDOM_SEED = 7
    
x_train, x_test, y_train, y_test = train_test_split(
    xdata, ydata, test_size=0.3, random_state=RANDOM_SEED, stratify=ydata
)

print(len(x_train), len(x_test), len(y_train), len(y_test))

# Change data shape to be correct for keras
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# Change data type to 32-bit float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Scale data values to be between 0 and 1
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert to one-hot labels 
num_classes = len(piece_to_label)  # Automatically determined
print(f"Number of classes: {num_classes}")

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Loaded 53 images
Piece types: {'black_bishop': 0, 'black_rook': 1, 'black_pawn': 2, 'black_knight': 3, 'black_king': 4, 'black_queen': 5}
Original image dimensions (estimated): 4032x3024
Resized image dimensions: 504x378
37 16 37 16
x_train shape: (37, 504, 378, 1)
37 train samples
16 test samples
Number of classes: 6
y_train shape: (37, 6)
y_test shape: (16, 6)


Build the model

In [13]:
# Prepares a model of a Neural Network that we will fit data to
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

/Users/lara.howe/.pyenv/versions/computer_vision/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Train the model using the training data 

In [14]:
# setup loss function (cross-entropy) and optimiser (adam)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])  # try 'fmeasure' for f1 score
print(model.summary())
# Conv2D layers: have biases by default
# each filter (height x width x input channels) -> scalar response
# accounting for padding too, out: (height (pad) x width (pad) x filters)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 502, 376, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 500, 374, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 250, 187, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 250, 187, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2992000)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │   382,976,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 382,995,718 (1.43 GB)

 Trainable params: 382,995,718 (1.43 GB)

 Non-trainable params: 0 (0.00 B)

None


# train the model!
batch_size = 128
epochs = 12

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Model evaluation 

In [15]:
# evaluate trained model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# make predictions on the test data
predictions = model.predict(x_test)
predictions = np.argmax(predictions,axis=1)

labels = np.argmax(y_test , axis=1)

Test loss: 1.7854732275009155
Test accuracy: 0.125
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
